# Assignment 15
This assignment is divided in two parts in which we are asked to model different queues.

## Part 1 - Performance indices of an M/G/1 queue
A server receives jobs according to a Poisson process of rate $\lambda$ = 3 $\frac{j}{s}$.

The duration of each job is distributed according to an Hyper-Exponential, of rates $\mu_{1}$ = 1 $\frac{j}{s}$ (prob. 0.2) and $\mu_{2}$ = 10 $\frac{j}{s}$ (prob. 0.8).

In [1]:
l_mg1 = 3
mu1_mg1 = 1
p_mg1 = 0.2
mu2_mg1 = 10

### Requests
- Compute the utilization of the system
- Compute the average number of jobs in the system
- Compute the average response time

### First and second moments of the service distribution
The first and second moments of the service distribution will be useful for the computation of the other performance indices.

In particular, the first moment of the service distribution will be (by definition of average service time) the performance index $D$.

The first moment of an hyper exponential can be computed as:
$$E[X_{hyp}] = \sum^{n}_{i=1}\frac{p_{i}}{\lambda_{i}}$$
While the second moment can be computed as:
$$E[X_{hyp}^{2}] = \sum^{n}_{i=1}\frac{2p_{i}}{\lambda_{i}^{2}}$$

In [2]:
debug = False
m1_mg1 = p_mg1/mu1_mg1 + (1-p_mg1)/mu2_mg1
D_mg1 = m1_mg1
m2_mg1 = 2*(p_mg1/mu1_mg1**2 + (1-p_mg1)/mu2_mg1**2)
if debug: print(m1_mg1, D_mg1, m2_mg1)

### Average Utilization
The computation of the average utilization of the system follows the utilization formula $$U=\lambda\cdot D = \rho$$

In [3]:
U_mg1 = l_mg1 * D_mg1
rho_mg1 = U_mg1

print("The average utilization of the system is {:.4f}".format(U_mg1))

The average utilization of the system is 0.8400


### Average Response time
The average response time of the system can be split up in multiple time frames 

$$R = D + W + w$$
Where:
- $D$ is the service time of the job
- $W$ is the time the job waits in queue
- $w$ is the remaining time for the currently active job to complete

The queue waiting time $W$ can be expressed in terms of the time $w$ $$W = \frac{\rho\cdot w}{1-\rho}$$

The remaining time for the currently acrtive job $w$ can be computed through the formula $$w = \frac{\lambda\cdot m_{2}}{2}$$

In [4]:
w_mg1 = l_mg1*m2_mg1/2
W_mg1 = rho_mg1*w_mg1/(1-rho_mg1)
R_mg1 = D_mg1 + W_mg1 + w_mg1

print("The average response time of the system is {:.4f}".format(R_mg1))

The average response time of the system is 4.1800


### Average number of jobs in the system
The average number of jobs in the system can be computed easily from the response time of the system through the application of Little's law.
$$N = R\cdot X = R\cdot\lambda$$

In [5]:
N_mg1 = R_mg1 * l_mg1

print("The average number of jobs in the system is {:.4f}".format(N_mg1))

The average number of jobs in the system is 12.5400


## Part 2 - Performance indices of an G/G/2 queue
A dual core server receives jobs with **inter-arrival time** distributed according to a **uniform distribution** between 0.1 and 0.2. 
The **duration of each job** is distributed according to an **Hyper-Exponential**, of rate $\mu_{1}$ = 1 $\frac{j}{s}$ (with prob. 0.2) and $\mu_{2}$ = 10 $\frac{j}{s}$ (with prob. 0.8).

In [6]:
a_gg2 = 0.1
b_gg2 = 0.2
mu1_gg2 = 1
mu2_gg2 = 10
p_gg2 = 0.2

### Requests
- Compute the utilization of the system
- Compute the average number of jobs in the system
- Compute the average response time

### Average Response time
Since the considered system is a G/G/2 queue, we'll need to start from the Response time and approximate its value with a special formula called the **Kingsman formula**.

$$R \cong D + \left[\frac{c_{a}^{2}+c_{v}^{2}}{2}\right]E[\Theta_{M/M/c}]$$

Where:
- $c_{a}$ and $c_{v}$ are the coefficients of variation of the inter arrival time distribution and the service time distribution, respectively

In [7]:
import math as m

m1_hyp_gg2 = p_gg2/mu1_gg2 + (1-p_gg2)/mu2_gg2
var_hyp_gg2 = 2*(p_gg2)/mu1_gg2**2+2*(1-p_gg2)/mu2_gg2**2 - m1_hyp_gg2**2

cv_gg2 = m.sqrt(var_hyp_gg2)/m1_hyp_gg2

m1_uni_gg2 = (a_gg2+b_gg2)/2
var_uni_gg2 = ((b_gg2 - a_gg2)**2)/12

ca_gg2 = m.sqrt(var_uni_gg2)/m1_uni_gg2

- $D$ is the average service time (or the first moment of the distribution of the service time)

In [8]:
D_gg2 = m1_hyp_gg2

- $\Theta_{M/M/c}$ is the expected waiting time in the corresponding M/M/c queue.

The arrival rate of the M/M/c queue to consider will be $\frac{1}{T}$, where $T$ is the average inter arrival time of the queue.  



In [9]:
l_mm2_gg2 = 1/m1_uni_gg2
rho_mm2_gg2 = l_mm2_gg2 * D_gg2/2

theta_mm2_gg2 = (D_gg2/(2*(1-rho_mm2_gg2))) / (1 + (1-rho_mm2_gg2)*(2/(2*rho_mm2_gg2)**2)*(1 + 2*rho_mm2_gg2))

In [10]:
R_gg2 = D_gg2 + (ca_gg2**2+cv_gg2**2)/2 * theta_mm2_gg2

print("The average response time of the system is (approximately) {:.4f}".format(R_gg2))

The average response time of the system is (approximately) 4.3895


### Utilization of the system

The average utilization of the system will be $$U = \frac{D \cdot \lambda}{2}$$

In this case, to compute our lambda we will use the inverse of the average inter arrival time of the system (just like if our arrivals were poisson distributed).

In [11]:
Uavg_gg2 = rho_mm2_gg2

print("The average utilization of the system will be {:.4f}".format(Uavg_gg2))

The average utilization of the system will be 0.9333


### Average Number of jobs in the system

The average number of jobs in the system will, as usual, be computed starting from Little's law.

In [12]:
N_gg2 = R_gg2 * l_mm2_gg2

print("The average number of jobs in the system is {:.4f}".format(N_gg2))

The average number of jobs in the system is 29.2635
